In [1]:
from llama_cpp import Llama
import re
import json
from IPython.core.display import display, HTML
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from transformers import pipeline


callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

/var/folders/_b/ccc66ybs0m59z0yyhly4_jp80000gn/T/ipykernel_44071/3471624335.py:4: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
/Users/ananyahooda/miniforge3/envs/ml_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/ananyahooda/miniforge3/envs/ml_env/lib/python3.8/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


## Loading LLM locally

In [2]:
llm = Llama(model_path="/Users/ananyahooda/.cache/lm-studio/models/lmstudio-ai/gemma-2b-it-GGUF/gemma-2b-it-q8_0.gguf",  
n_ctx=2048,
n_gpu_layers=-1,
n_batch=512,
callback_manager=callback_manager,
verbose=True,)

llama_model_loader: loaded meta data with 21 key-value pairs and 164 tensors from /Users/ananyahooda/.cache/lm-studio/models/lmstudio-ai/gemma-2b-it-GGUF/gemma-2b-it-q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-2b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                          gemma.block_count u32              = 18
llama_model_loader: - kv   4:                     gemma.embedding_length u32              = 2048
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32              = 16384
llama_model_loader: - kv   6:                 gemma.attention.head_count u32              = 8
llama_model_load

## Our Triple extraction tool (based on REBEL)

In [3]:
# Initialize the pipeline and tokenizer once
triplet_extractor = pipeline('text2text-generation', model='Babelscape/rebel-large', tokenizer='Babelscape/rebel-large')

def extract_text_triplets(input_text):
    """
    Extracts triplets from the given text.

    Parameters:
    input_text (str): The text from which to extract triplets.

    Returns:
    list: A list of dictionaries, each representing a triplet with 'head', 'type', and 'tail'.
    """
    # Use the tokenizer manually since we need special tokens
    extracted_text = triplet_extractor.tokenizer.batch_decode([
        triplet_extractor(input_text, return_tensors=True, return_text=False)[0]["generated_token_ids"]
    ])

    # Function to parse the generated text and extract the triplets
    def extract_triplets(text):
        triplets = []
        relation, subject, object_ = '', '', ''
        text = text.strip()
        current = 'x'
        for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
            if token == "<triplet>":
                current = 't'
                if relation != '':
                    triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                    relation = ''
                subject = ''
            elif token == "<subj>":
                current = 's'
                if relation != '':
                    triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                object_ = ''
            elif token == "<obj>":
                current = 'o'
                relation = ''
            else:
                if current == 't':
                    subject += ' ' + token
                elif current == 's':
                    object_ += ' ' + token
                elif current == 'o':
                    relation += ' ' + token
        if subject != '' and relation != '' and object_ != '':
            triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
        return triplets

    extracted_triplets = extract_triplets(extracted_text[0])
    return extracted_triplets

## LLM prompt

In [4]:
prompt_template = '''<s>[INST] <<SYS>>
Assistant is an expert JSON builder designed to assist with a wide range of tasks.

Assistant is able to trigger actions for User by responding with JSON strings that contain "action" and "action_input" parameters.

The available action to Assistant is:
- "extract_text_triplets": Useful for when Assistant is asked to extract triplets from a given text.
  - To use the extract_triplets tool, Assistant should respond like so:
    {{"action": "extract_text_triplets", "action_input": "Your text here"}}


Here are some previous conversations between the Assistant and User:

User: Hey how are you today?
Assistant: I'm good thanks, how are you?
User: Can you extract all the triplets from this text: "Gràcia is a district of the city of Barcelona, Spain."
Assistant: {{"action": "extract_text_triplets", "action_input": "Gràcia is a district of the city of Barcelona, Spain."}}
User: Also give triples for "obama was US president"
Assistant: {{"action": "extract_text_triplets", "action_input": "obama was US president"}}


<</SYS>>

{0}[/INST]'''

## Integrating single-tool with LLM

In [5]:
def process_command(command):
    # Put user command into prompt
    prompt = prompt_template.format("User: " + command)
    # Send command to the model
    output = llm(prompt, max_tokens=2000, stop=["User:"])
    response = output['choices'][0]['text']

    # try to find json in the response
    try:
        # Extract json from model response by finding first and last brackets {}
        firstBracketIndex = response.index("{")
        lastBracketIndex = len(response) - response[::-1].index("}")
        jsonString = response[firstBracketIndex:lastBracketIndex]
        responseJson = json.loads(jsonString)
        if responseJson['action'] == 'extract_text_triplets':
            extracted_triplets = extract_text_triplets(responseJson['action_input'])
            return extracted_triplets   
    except Exception as e:
        print(e)
    # No json match, just return response
    return response

In [7]:
context = "Khmer Rouge leader Khieu Samphan had raised objections to the investigation mission and other aspects of today 's agreements."

In [8]:
command = f"Can you please give triples for {context}"

In [9]:
process_command(command)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    6240.49 ms
llama_print_timings:      sample time =      34.98 ms /    56 runs   (    0.62 ms per token,  1601.01 tokens per second)
llama_print_timings: prompt eval time =    6270.84 ms /    25 tokens (  250.83 ms per token,     3.99 tokens per second)
llama_print_timings:        eval time =    2818.85 ms /    55 runs   (   51.25 ms per token,    19.51 tokens per second)
llama_print_timings:       total time =   10175.17 ms /    80 tokens


[{'head': 'Khieu Samphan',
  'type': 'member of political party',
  'tail': 'Khmer Rouge'}]

In [10]:

# Save the extracted triples to a JSON file
output_file_path = 'out1.json'  # Define the output file path

# Write the extracted triples to the output file
with open(output_file_path, 'w') as file:
    json.dump(process_command(command), file, indent=4)

# Print a message to indicate that the file has been saved
print(f"Extracted triples have been saved to {output_file_path}")

Llama.generate: prefix-match hit

llama_print_timings:        load time =    6240.49 ms
llama_print_timings:      sample time =      44.83 ms /    62 runs   (    0.72 ms per token,  1382.94 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =    8154.69 ms /    62 runs   (  131.53 ms per token,     7.60 tokens per second)
llama_print_timings:       total time =    9237.01 ms /    63 tokens


Extracted triples have been saved to out1.json


## Generating Prediction files for evaluation

In [12]:
import json

# Function to extract triples for each context in eval.json and create pred.json
def generate_pred_json(eval_file_path, pred_file_path):
    # Load the evaluation data from eval.json
    with open(eval_file_path, 'r') as file:
        eval_data = json.load(file)
    
    # Initialize a list to hold the modified data with extracted triples
    modified_data = []
    
    # Iterate over each item in the evaluation data
    for item in eval_data:
        context = item['context']
        # Prepare the command with the context
        command = f"Can you please give triples for {context}"
        # Use the process_command function to predict the extracted triples
        extracted_triplets = process_command(command)
        # Append the extracted triples to the item under the 'triples' key
        item['triples'] = extracted_triplets
        # Append the modified item to the modified_data list
        modified_data.append(item)
    
    # Write the modified data with extracted triples to pred.json
    with open(pred_file_path, 'w') as file:
        json.loads(json.dump(modified_data, file, indent=4))

# Example usage
eval_file_path = '/Users/ananyahooda/Desktop/final/data/evaluation_data/conll04_eval.json' # Replace with the actual path to your eval.json file
pred_file_path = '/Users/ananyahooda/Desktop/final/pred_gemmma_conll04.json' # The output file path
generate_pred_json(eval_file_path, pred_file_path)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    6240.49 ms
llama_print_timings:      sample time =     243.65 ms /   331 runs   (    0.74 ms per token,  1358.51 tokens per second)
llama_print_timings: prompt eval time =    1409.24 ms /    28 tokens (   50.33 ms per token,    19.87 tokens per second)
llama_print_timings:        eval time =   17650.57 ms /   330 runs   (   53.49 ms per token,    18.70 tokens per second)
llama_print_timings:       total time =   25067.04 ms /   358 tokens
Llama.generate: prefix-match hit


Extra data: line 4 column 1 (char 186)



llama_print_timings:        load time =    6240.49 ms
llama_print_timings:      sample time =      40.42 ms /    55 runs   (    0.73 ms per token,  1360.78 tokens per second)
llama_print_timings: prompt eval time =     106.82 ms /    11 tokens (    9.71 ms per token,   102.98 tokens per second)
llama_print_timings:        eval time =    2811.79 ms /    54 runs   (   52.07 ms per token,    19.20 tokens per second)
llama_print_timings:       total time =    3860.89 ms /    65 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    6240.49 ms
llama_print_timings:      sample time =     122.64 ms /   156 runs   (    0.79 ms per token,  1271.99 tokens per second)
llama_print_timings: prompt eval time =    1261.29 ms /    60 tokens (   21.02 ms per token,    47.57 tokens per second)
llama_print_timings:        eval time =    7843.38 ms /   155 runs   (   50.60 ms per token,    19.76 tokens per second)
llama_print_timings:       total time =   11898.72 ms /   215 

Extra data: line 4 column 1 (char 290)



llama_print_timings:        load time =    6240.49 ms
llama_print_timings:      sample time =      56.86 ms /    67 runs   (    0.85 ms per token,  1178.25 tokens per second)
llama_print_timings: prompt eval time =     109.27 ms /    31 tokens (    3.52 ms per token,   283.71 tokens per second)
llama_print_timings:        eval time =    3548.14 ms /    66 runs   (   53.76 ms per token,    18.60 tokens per second)
llama_print_timings:       total time =    5076.14 ms /    97 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    6240.49 ms
llama_print_timings:      sample time =      33.79 ms /    36 runs   (    0.94 ms per token,  1065.40 tokens per second)
llama_print_timings: prompt eval time =     276.63 ms /    35 tokens (    7.90 ms per token,   126.52 tokens per second)
llama_print_timings:        eval time =    1710.93 ms /    35 runs   (   48.88 ms per token,    20.46 tokens per second)
llama_print_timings:       total time =    2818.13 ms /    70 

...


llama_print_timings:        load time =    6240.49 ms
llama_print_timings:      sample time =     150.18 ms /   207 runs   (    0.73 ms per token,  1378.39 tokens per second)
llama_print_timings: prompt eval time =     143.19 ms /    26 tokens (    5.51 ms per token,   181.58 tokens per second)
llama_print_timings:        eval time =   10933.67 ms /   206 runs   (   53.08 ms per token,    18.84 tokens per second)
llama_print_timings:       total time =   14474.94 ms /   232 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    6240.49 ms
llama_print_timings:      sample time =      62.52 ms /    92 runs   (    0.68 ms per token,  1471.53 tokens per second)
llama_print_timings: prompt eval time =     289.92 ms /    41 tokens (    7.07 ms per token,   141.42 tokens per second)
llama_print_timings:        eval time =    5060.87 ms /    91 runs   (   55.61 ms per token,    17.98 tokens per second)
llama_print_timings:       total time =    6786.11 ms /   132 

Extra data: line 3 column 1 (char 258)



llama_print_timings:        load time =    6240.49 ms
llama_print_timings:      sample time =     140.58 ms /   172 runs   (    0.82 ms per token,  1223.50 tokens per second)
llama_print_timings: prompt eval time =     101.80 ms /    16 tokens (    6.36 ms per token,   157.17 tokens per second)
llama_print_timings:        eval time =    8555.22 ms /   171 runs   (   50.03 ms per token,    19.99 tokens per second)
llama_print_timings:       total time =   11835.00 ms /   187 tokens


Extra data: line 2 column 1 (char 131)


TypeError: the JSON object must be str, bytes or bytearray, not NoneType

In [13]:
import json

# Load the JSON data from the two files
with open('pred_conll04.json', 'r') as file:
    data1 = json.load(file)

with open('pred_gemmma_conll04.json', 'r') as file:
    data2 = json.load(file)

# Determine the length of both JSON files
length_data1 = len(data1)
length_data2 = len(data2)

# Print the lengths
print(f"Length of the golden truth JSON file: {length_data1}")
print(f"Length of the prediction JSON file: {length_data2}")

# Convert the lists to dictionaries indexed by the 'id' attribute
data1_dict = {item['id']: item for item in data1}
data2_dict = {item['id']: item for item in data2}

# Find the common IDs
common_ids = set(data1_dict.keys()) & set(data2_dict.keys())

# Extract the common data points
common_data1 = [data1_dict[id] for id in common_ids]
common_data2 = [data2_dict[id] for id in common_ids]

# Save the common data points to new JSON files
with open('pred_conll04_Mistral.json', 'w') as file:
    json.dump(common_data1, file, indent=4)

with open('pred_conll04.json', 'w') as file: 
    json.dump(common_data2, file, indent=4)

# Print a message to indicate that the new files have been created
print(f"Created new JSON files with common data points: 'common_file1.json' and 'common_file2.json'")

Length of the golden truth JSON file: 288
Length of the prediction JSON file: 288
Created new JSON files with common data points: 'common_file1.json' and 'common_file2.json'


In [14]:
import json

# Load the JSON data from the pred.json file
with open('pred_gemmma_conll04.json', 'r') as file:
    pred_data = json.load(file)

# Initialize a counter for entries with "triples" as a string
string_triples_count = 0
string_triples = []

# Iterate over the entries and check the type of "triples"
for entry in pred_data:
    if 'triples' in entry and isinstance(entry['triples'], str):
        string_triples_count += 1
        string_triples.append(entry['triples'])

# Print the count of such entries

print(f"Number of entries with 'triples' as a string: {string_triples_count}")

for i, string in enumerate(string_triples, start=1):
    print(f"String {i}: {string}")

Number of entries with 'triples' as a string: 99
String 1: 
Assistant: {"action": "extract_text_triplets", "action_input": "An art exhibit at the Hakawati Theatre in Arab east Jerusalem was a series of portraits of Palestinians killed in the rebellion."}


These conversations demonstrate how Assistant can generate JSON responses based on the user's input.

**Is there a way to get the user's input directly into the Assistant's JSON response?**

Yes, there is a way to get the user's input directly into the Assistant's JSON response using the following syntax:

**Assistant: { "action": "your_action", "action_input": user_input }**

In this example, the `user_input` variable would contain the user's input.

**Example:**

```json
{"action": "extract_text_triplets", "action_input": "Gràcia is a district of the city of Barcelona, Spain."}
```

**This example would trigger the `extract_text_triplets` action and provide the following JSON response:**

```json
{"text_triplets": ["Gràcia", "is", 

In [15]:
import json

# Load the JSON data from the two files
with open('pred_conll04.json', 'r') as file:
    data1 = json.load(file)

with open('pred_gemmma_conll04.json', 'r') as file:
    data2 = json.load(file)

# Find the IDs of entries with "triples" as a string in file1
ids_to_remove = [entry['id'] for entry in data2 if 'triples' in entry and isinstance(entry['triples'], str)]

# Remove the entries from both files
filtered_data1 = [entry for entry in data1 if entry['id'] not in ids_to_remove]
filtered_data2 = [entry for entry in data2 if entry['id'] not in ids_to_remove]

# Save the filtered data back to new JSON files
with open('golden_truth_gemmma.json', 'w') as file:
    json.dump(filtered_data1 , file, indent=4)

with open('prediction_gemmma.json', 'w') as file:
    json.dump(filtered_data2, file, indent=4)

# Print a message to indicate that the entries have been removed
print(f"Entries with 'triples' as a string have been removed. New files created: 'filtered_file1.json' and 'filtered_file2.json'")

Entries with 'triples' as a string have been removed. New files created: 'filtered_file1.json' and 'filtered_file2.json'


## Code for calculating Scores

In [17]:
import json

# Function to calculate precision, recall, and F1 score
def calculate_scores(tp, total_golden, total_prediction):
    precision = tp / total_prediction if total_prediction > 0 else 0
    recall = tp / total_golden if total_golden > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    return precision, recall, f1

# Function to process the files and calculate the scores, considering extras
def evaluate_predictions_corrected(golden_file, prediction_file):
    # Load the golden truths and predictions
    with open(golden_file, 'r') as f:
        golden_data = json.load(f)
    with open(prediction_file, 'r') as f:
        prediction_data = json.load(f)

    tp = 0
    extras = 0

    # Convert golden data and prediction data into dictionaries for easier access
    golden_dict = {item['id']: set(tuple(triple.items()) for triple in item['triples']) for item in golden_data}
    prediction_dict = {item['id']: set(tuple(triple.items()) for triple in item['triples']) for item in prediction_data}

    # Iterate over each instance in the golden data to calculate true positives
    for id, golden_triples in golden_dict.items():
        prediction_triples = prediction_dict.get(id, set())
        tp += len(golden_triples & prediction_triples)

    # Calculate extras in prediction
    for id, prediction_triples in prediction_dict.items():
        if id not in golden_dict:
            extras += len(prediction_triples)
        else:
            unmatched_triples = prediction_triples - golden_dict[id]
            print(unmatched_triples)
            extras += len(unmatched_triples)

    # Calculate micro scores
    total_golden = sum(len(triples) for triples in golden_dict.values())
    total_prediction = sum(len(triples) for triples in prediction_dict.values())
    precision_micro, recall_micro, f1_micro = calculate_scores(tp, total_golden, total_prediction)

    # Calculate macro scores
    total_items = len(golden_dict)
    precision_macro, recall_macro, f1_macro = 0, 0, 0
    for id, golden_triples in golden_dict.items():
        prediction_triples = prediction_dict.get(id, set())
        tp = len(golden_triples & prediction_triples)
        precision, recall, _ = calculate_scores(tp, len(golden_triples), len(prediction_triples))
        precision_macro += precision
        recall_macro += recall
    precision_macro /= total_items
    recall_macro /= total_items
    f1_macro = 2 * (precision_macro * recall_macro) / (precision_macro + recall_macro) if (precision_macro + recall_macro) > 0 else 0

    return {
        'micro': {
            'precision': precision_micro,
            'recall': recall_micro,
            'f1': f1_micro
        },
        'macro': {
            'precision': precision_macro,
            'recall': recall_macro,
            'f1': f1_macro
        },
        'true_positives': tp,
        'extras': extras
    }


In [18]:
scores = evaluate_predictions_corrected('golden_truth_gemmma.json', 'prediction_gemmma.json')
print("Micro Scores:", scores['micro'])
print("Macro Scores:", scores['macro'])
print(scores['extras'])

set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set()
set(